<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    """Calculate effective frequency from generated flash times."""
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency."""
    F = min(target_Fe * 1.2, max_freq)  # Start slightly higher
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"⚠️ Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Formats a single STP line with all four oscillator parameter blocks concatenated.
    Each block: step duration, wave type, start freq, end freq, start duty, end duty, LED config (4 values), start intensity, end intensity.
    The led_assignments parameter should be a list of four lists, one per oscillator.
    """
    # Build each oscillator's parameter block:
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        # Here we use the same frequency, duty, and brightness for both start and end values.
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    # Combine all four blocks into one STP line (separated by commas, no extra 'STP' tokens in between)
    return f'STP"{",".join(blocks)}"'

def generate_strobe_sequence():
    """Generates a strobe sequence file with proper STP formatting for all four oscillators in one line per step."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    # We now assume the default LED assignment as per the creator's info:
    # Osc1: 1,0,0,0; Osc2: 0,1,0,0; Osc3: 0,0,1,0; Osc4: 0,0,0,1.
    led_config = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]]

    # Header lines
    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    # Determine number of steps (simple approximation)
    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    # Generate flash times based on stimulation type
    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    # For each flash time, generate one STP line (which includes the parameters for all 4 oscillators)
    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        # For simplicity, use start values for the first half and end values for the second half:
        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"✅ Generated {file_name} with correct formatting.")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
✅ Generated strobe_periodic.txt with correct formatting.
